# 基于Transformers的阅读理解实现

## step1 导入包

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator

## step2 加载数据集

In [4]:
datasets = load_dataset('cmrc2018', cache_dir='./data')
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1002
    })
})

In [5]:
print(datasets['train'][0])

{'id': 'TRAIN_186_QUERY_0', 'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。', 'question': '范廷颂是什么时候被任为主教的？', 'answers': {'text': ['1963年'], 'answer_start': [30]}}


## step3 数据集预处理

In [6]:
tokenizer = AutoTokenizer.from_pretrained('D:/pretrained_model/models--hfl--chinese-macbert-base')

In [7]:
print(datasets['train'][0])

{'id': 'TRAIN_186_QUERY_0', 'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。', 'question': '范廷颂是什么时候被任为主教的？', 'answers': {'text': ['1963年'], 'answer_start': [30]}}


In [8]:
sample_dataset = datasets['train'].select(range(2))
sample_dataset

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 2
})

In [9]:
tokenizer_examples = tokenizer(text=sample_dataset['question'],
                               text_pair=sample_dataset['context'],
                               max_length=512,
                               truncation='only_second',
                               return_offsets_mapping=True,
                               padding='max_length')

In [10]:
tokenizer_examples.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping'])

In [11]:
print(tokenizer_examples['offset_mapping'][0], len(tokenizer_examples['offset_mapping'][0]))

[(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 45), (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (62, 63), (63, 67), (67, 68), (68, 69), (69, 70), (70, 71), (71, 72), (72, 73), (73, 74), (74, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 81), (81, 82), (82, 83), (83, 84), (84, 85), (85, 86), (86, 87), (87, 91), (91, 92), (92, 93), (93, 94), (94, 95), (95, 96), (96, 97), (97, 98), (98, 99), (

In [12]:
offset_mapping = tokenizer_examples.pop("offset_mapping")

In [13]:
print(tokenizer_examples)

{'input_ids': [[101, 5745, 2455, 7563, 3221, 784, 720, 3198, 952, 6158, 818, 711, 712, 3136, 4638, 8043, 102, 5745, 2455, 7563, 3364, 3322, 8020, 8024, 8021, 8024, 1760, 1399, 924, 4882, 185, 5735, 4449, 8020, 8021, 8024, 3221, 6632, 1298, 5384, 7716, 1921, 712, 3136, 3364, 3322, 511, 9155, 2399, 6158, 818, 711, 712, 3136, 8039, 8431, 2399, 6158, 3091, 1285, 711, 1921, 712, 3136, 3777, 1079, 2600, 3136, 1277, 2134, 2429, 5392, 4415, 8039, 8447, 2399, 6158, 3091, 1285, 711, 2600, 712, 3136, 8024, 1398, 2399, 2399, 2419, 6158, 3091, 1285, 711, 3364, 3322, 8039, 8170, 2399, 123, 3299, 4895, 686, 511, 5745, 2455, 7563, 754, 9915, 2399, 127, 3299, 8115, 3189, 1762, 6632, 1298, 2123, 2398, 4689, 1921, 712, 3136, 1355, 5683, 3136, 1277, 1139, 4495, 8039, 4997, 2399, 3198, 2970, 1358, 5679, 1962, 3136, 5509, 1400, 8024, 6158, 671, 855, 6632, 1298, 4868, 4266, 2372, 1168, 3777, 1079, 5326, 5330, 1071, 2110, 689, 511, 5745, 2455, 7563, 754, 9211, 2399, 1762, 3777, 1079, 1920, 934, 6887, 7368, 21

In [14]:
tokenizer_examples.sequence_ids(0)

[None,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 None,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1

In [15]:
offset_mapping, len(offset_mapping)

([[(0, 0),
   (0, 1),
   (1, 2),
   (2, 3),
   (3, 4),
   (4, 5),
   (5, 6),
   (6, 7),
   (7, 8),
   (8, 9),
   (9, 10),
   (10, 11),
   (11, 12),
   (12, 13),
   (13, 14),
   (14, 15),
   (0, 0),
   (0, 1),
   (1, 2),
   (2, 3),
   (3, 4),
   (4, 5),
   (5, 6),
   (6, 7),
   (7, 8),
   (8, 9),
   (9, 10),
   (10, 11),
   (11, 12),
   (12, 13),
   (13, 14),
   (14, 15),
   (15, 16),
   (16, 17),
   (17, 18),
   (18, 19),
   (19, 20),
   (20, 21),
   (21, 22),
   (22, 23),
   (23, 24),
   (24, 25),
   (25, 26),
   (26, 27),
   (27, 28),
   (28, 29),
   (29, 30),
   (30, 34),
   (34, 35),
   (35, 36),
   (36, 37),
   (37, 38),
   (38, 39),
   (39, 40),
   (40, 41),
   (41, 45),
   (45, 46),
   (46, 47),
   (47, 48),
   (48, 49),
   (49, 50),
   (50, 51),
   (51, 52),
   (52, 53),
   (53, 54),
   (54, 55),
   (55, 56),
   (56, 57),
   (57, 58),
   (58, 59),
   (59, 60),
   (60, 61),
   (61, 62),
   (62, 63),
   (63, 67),
   (67, 68),
   (68, 69),
   (69, 70),
   (70, 71),
   (71, 72),
  

In [16]:
# 处理截断的过程中，答案被截断的情况

for idx, offset in enumerate(offset_mapping):
    answer = sample_dataset[idx]['answers']
    start_char = answer['answer_start'][0]
    end_char = start_char + len(answer['text'][0])

    # 定位答案再token中的起始位置和结束位置
    # 一种策略，我们要拿到context的起始和终止位置，然后左右逼近
    # sequence_ids(idx) 方法返回一个列表，用于指示第 idx 个示例的每个分词后的 token 属于哪个输入序列。

    context_start = tokenizer_examples.sequence_ids(idx).index(1)
    context_end = tokenizer_examples.sequence_ids(idx).index(None, context_start) - 1 # 从context_start开始查找第一个出现的None

    # 判断答案不在上下文中
    if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
        start_token_pos = 0
        end_token_pos = 0
    else:
        # 找到包含答案起始位置的上下文的开头
        token_id = context_start
        while token_id <= context_end and offset[token_id][0] < start_char:
            token_id += 1
        start_token_pos = token_id

        # 找到包含答案终止位置的上下文的结束
        token_id = context_end
        while token_id >= context_start and offset[token_id][1] > end_char:
            token_id -= 1
        end_token_pos = token_id
    
    print(answer, start_char, end_char, context_start, context_end, start_token_pos, end_token_pos)
    print("token answer decode:", tokenizer.decode(tokenizer_examples['input_ids'][idx][start_token_pos : end_token_pos + 1]))
    

{'text': ['1963年'], 'answer_start': [30]} 30 35 17 510 47 48
token answer decode: 1963 年
{'text': ['1990年被擢升为天主教河内总教区宗座署理'], 'answer_start': [41]} 41 62 15 510 53 70
token answer decode: 1990 年 被 擢 升 为 天 主 教 河 内 总 教 区 宗 座 署 理


In [17]:
def process_function(examples):
    tokenizer_examples = tokenizer(text=examples['question'],
                                    text_pair=examples['context'],
                                    max_length=512,
                                    truncation='only_second',
                                    return_offsets_mapping=True,
                                    padding='max_length'
                                   )
    offset_mapping = tokenizer_examples.pop('offset_mapping')

    # 因为后面会使用到tokenizer batch的方式进行处理
    start_position = []
    end_position = []
    for idx, offset in enumerate(offset_mapping):
        answer = examples['answers'][idx]
        start_char = answer['answer_start'][0]
        end_char = start_char + len(answer['text'][0])

        # 定位答案再token中的起始位置和结束位置
        # 一种策略，我们要拿到context的起始和终止位置，然后左右逼近
        # sequence_ids(idx) 方法返回一个列表，用于指示第 idx 个示例的每个分词后的 token 属于哪个输入序列。

        context_start = tokenizer_examples.sequence_ids(idx).index(1)
        context_end = tokenizer_examples.sequence_ids(idx).index(None, context_start) - 1 # 从context_start开始查找第一个出现的None

        # 判断答案不在上下文中
        if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
            start_token_pos = 0
            end_token_pos = 0
        else:
            # 找到包含答案起始位置的上下文的开头
            token_id = context_start
            while token_id <= context_end and offset[token_id][0] < start_char:
                token_id += 1
            start_token_pos = token_id

            # 找到包含答案终止位置的上下文的结束
            token_id = context_end
            while token_id >= context_start and offset[token_id][1] > end_char:
                token_id -= 1
            end_token_pos = token_id
        start_position.append(start_token_pos)
        end_position.append(end_token_pos)
    
    tokenizer_examples['start_positions'] = start_position
    tokenizer_examples['end_positions'] = end_position
    return tokenizer_examples


In [18]:
datasets['train']

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 10142
})

In [19]:
tokenizer_datasets = datasets.map(function=process_function, batched=True, remove_columns=datasets['train'].column_names)
tokenizer_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 1002
    })
})

## step4 创建模型

In [20]:


model = AutoModelForQuestionAnswering.from_pretrained('D:/pretrained_model/models--hfl--chinese-macbert-base')

args = TrainingArguments(
    output_dir='./models_for_qa',
    per_device_eval_batch_size=32,
    per_device_train_batch_size=4,
    evaluation_strategy='epoch',
    save_strategy="epoch",
    logging_steps=100,
    load_best_model_at_end=True,
    num_train_epochs=1
)

trainer = Trainer(
    model=model,
    args = args,
    tokenizer = tokenizer,
    train_dataset= tokenizer_datasets['validation'],
    eval_dataset= tokenizer_datasets['test'],
    data_collator=DefaultDataCollator()
)

trainer.train()

d:\anaconda3\envs\py39\lib\site-packages\transformers\modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="c

{'loss': 3.0902, 'learning_rate': 4.3788819875776396e-05, 'epoch': 0.12}


 25%|██▍       | 200/805 [01:06<03:15,  3.09it/s]

{'loss': 1.704, 'learning_rate': 3.7577639751552796e-05, 'epoch': 0.25}


 37%|███▋      | 300/805 [01:38<02:52,  2.92it/s]

...

{'loss': 1.0959, 'learning_rate': 6.521739130434783e-06, 'epoch': 0.87}


 99%|█████████▉| 800/805 [04:13<00:01,  3.30it/s]

{'loss': 0.9661, 'learning_rate': 3.1055900621118013e-07, 'epoch': 0.99}


100%|██████████| 805/805 [04:15<00:00,  3.59it/s]

KeyboardInterrupt: 

## step9 模型推理

In [ ]:
from transformers import pipeline

pipe = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)

pipe(question="小明在哪里上班？", context="小明在北京上班。")

d:\anaconda3\envs\py39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'model' is not defined